In [1]:
import torch
import pickle
import argparse
from tqdm import tqdm
from collections import defaultdict
from multiprocessing import Pool, Manager
from os.path import basename, join, dirname, abspath


import sys
sys.path.append(abspath('../src'))
import model.util as util
from model.rnn import RNNEncoder
from model.bow import BOWEncoder


# load checkpoint
checkpoint = torch.load('../checkpoint/felix-e500-v30000_h512_p20_l2_d0.0-0.0_b16_gru_Adam_lr0.0001', map_location=lambda storage, loc: storage)

# load data
model_args = checkpoint['args']
with open(model_args.data, 'rb') as rf:
    data = pickle.load(rf)
dic_embed = data['dic_embed']
def_embed = data['def_embed']
dic_word2ix = data['dic_word2ix']
def_word2ix = data['def_word2ix']
dic_ix2word = {v: k for k, v in dic_word2ix.items()}
rvd_candidates = set(data['rvd_candidates'])
print('[1] load data OK:', model_args.data)

# load model
model_args.gpu = -1
encoder = RNNEncoder(def_word2ix, model_args)
encoder.load_state_dict(checkpoint['state_dict'])
print('[2] load model OK!')

dic_embed = util.normalize_matrix_by_row(dic_embed.numpy())

[1] load data OK: ../data/felix-e500-v30000.pk
[2] load model OK!


In [2]:
def rank_sen(sen):
    out_embs = encoder.estimate_from_strsens([sen], normalize=True)
    simi_matrix = out_embs.dot(dic_embed.transpose()).reshape(-1)
    rank = sorted(range(len(dic_word2ix)), key=lambda i: simi_matrix[i], reverse=True)
    return [dic_ix2word[i] for i in rank[:20]]

In [3]:
encoder.estimate_from_strsens(['a fruit', 'hello', 'fwea fwea']).shape

(3, 500)

In [23]:
e = RNNEncoder({}, model_args)

RuntimeError: While copying the parameter named embed.weight, whose dimensions in the model are torch.Size([]) and whose dimensions in the checkpoint are torch.Size([30003, 500]).